In [1]:
from IPython.display import display, HTML
display(HTML("""
<style>
div.container{width:85% !important;}
div.cell.code_cell.rendered{width:100%;}
div.input_prompt{padding:0px;}
div.CodeMirror {font-family:Consolas; font-size:12pt;}
div.output {font-size:12pt; font-weight:bold;}
div.input {font-family:Consolas; font-size:12pt;}
div.prompt {min-width:70px;}
div#toc-wrapper{padding-top:120px;}
div.text_cell_render ul li{font-size:12pt;padding:5px;}
table.dataframe{font-size:12px;}
</style>
"""))

In [4]:
import os # 모델 저장할 폴더가 있는지 여부와 폴더 생성
import numpy as np
import pandas as pd # read_csv, get_dummies, crosstab
import seaborn as sns # iris 데이터를 df로 가져오기
from sklearn import datasets # iris 데이터(X, y) 배열로 가져오기
from sklearn.preprocessing import LabelEncoder # 라벨 인코더
from tensorflow.keras.utils import to_categorical # 원핫인코딩
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential, save_model, load_model
from tensorflow.keras.layers import Input, Dense, Dropout, LeakyReLU
from tensorflow.keras.metrics import Recall, Precision # compile 단계에서 사용
from tensorflow.keras.callbacks import EarlyStopping, Callback, ModelCheckpoint
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix

# 1. 기본적인 DNN

In [9]:
# 1. 데이터 생성 및 전처리
iris = datasets.load_iris()
iris_X = iris.data
iris_y = iris.target # label encoding 이미 됨
train_X, test_X, train_y, test_y = train_test_split(iris_X, iris_y,
                                                   test_size=0.2,
                                                   stratify=iris_y)
train_X.shape, train_y.shape, test_X.shape, test_y.shape

((120, 4), (120,), (30, 4), (30,))

In [10]:
# 2. 모델구성
model = Sequential([
    Input(4),
    Dense(units=64, activation='relu'),
    Dense(units=128, activation='relu'),
    Dropout(0.2), 
    Dense(units=50, activation='relu'),
    Dropout(0.2),
    Dense(units=30, activation='relu'),
    Dropout(0.1),
    Dense(units=3, activation='softmax')    
])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 64)                320       
                                                                 
 dense_1 (Dense)             (None, 128)               8320      
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                                 
 dense_2 (Dense)             (None, 50)                6450      
                                                                 
 dropout_1 (Dropout)         (None, 50)                0         
                                                                 
 dense_3 (Dense)             (None, 30)                1530      
                                                                 
 dropout_2 (Dropout)         (None, 30)                0

In [12]:
# 3. 학습과정 설정
model.compile(loss='sparse_categorical_crossentropy',
             optimizer='adam',
              metrics=['accuracy']) # metrics.Recall(), metrics.Precision()은 원핫인코딩 하여 2차원으로 넣어야!!
# 4. 모델 학습
earlystopping = EarlyStopping(patience=40)# monitor='val_loss' 기본값
hist=model.fit(train_X,train_y,batch_size=40,
               epochs=200,
               validation_split=0.2,
               callbacks=[earlystopping])

Epoch 1/200
3/3 [==============================] - 1s 77ms/step - loss: 1.1361 - accuracy: 0.3438 - val_loss: 1.0626 - val_accuracy: 0.3333
Epoch 2/200
3/3 [==============================] - 0s 14ms/step - loss: 1.0985 - accuracy: 0.3750 - val_loss: 1.0375 - val_accuracy: 0.5417
Epoch 3/200
3/3 [==============================] - 0s 17ms/step - loss: 1.0519 - accuracy: 0.4479 - val_loss: 1.0198 - val_accuracy: 0.5417
Epoch 4/200
3/3 [==============================] - 0s 14ms/step - loss: 0.9954 - accuracy: 0.5729 - val_loss: 0.9864 - val_accuracy: 0.6250
Epoch 5/200
3/3 [==============================] - 0s 12ms/step - loss: 0.9815 - accuracy: 0.5938 - val_loss: 0.9367 - val_accuracy: 0.7083
Epoch 6/200
3/3 [==============================] - 0s 13ms/step - loss: 0.9471 - accuracy: 0.6562 - val_loss: 0.8822 - val_accuracy: 0.9167
Epoch 7/200
3/3 [==============================] - 0s 13ms/step - loss: 0.9145 - accuracy: 0.6979 - val_loss: 0.8176 - val_accuracy: 0.9167
Epoch 8/200
3/3 [===

3/3 [==============================] - 0s 12ms/step - loss: 0.1537 - accuracy: 0.9583 - val_loss: 0.0369 - val_accuracy: 1.0000
Epoch 60/200
3/3 [==============================] - 0s 13ms/step - loss: 0.0847 - accuracy: 0.9583 - val_loss: 0.0851 - val_accuracy: 0.9167
Epoch 61/200
3/3 [==============================] - 0s 13ms/step - loss: 0.0955 - accuracy: 0.9479 - val_loss: 0.1912 - val_accuracy: 0.9167
Epoch 62/200
3/3 [==============================] - 0s 16ms/step - loss: 0.1249 - accuracy: 0.9479 - val_loss: 0.1033 - val_accuracy: 0.9167
Epoch 63/200
3/3 [==============================] - 0s 13ms/step - loss: 0.0780 - accuracy: 0.9792 - val_loss: 0.0456 - val_accuracy: 1.0000
Epoch 64/200
3/3 [==============================] - 0s 12ms/step - loss: 0.0465 - accuracy: 0.9896 - val_loss: 0.0518 - val_accuracy: 1.0000
Epoch 65/200
3/3 [==============================] - 0s 10ms/step - loss: 0.0366 - accuracy: 0.9896 - val_loss: 0.0928 - val_accuracy: 0.9167
Epoch 66/200
3/3 [========

3/3 [==============================] - 0s 17ms/step - loss: 0.0211 - accuracy: 0.9896 - val_loss: 0.0222 - val_accuracy: 1.0000
Epoch 118/200
3/3 [==============================] - 0s 12ms/step - loss: 0.0641 - accuracy: 0.9792 - val_loss: 0.0216 - val_accuracy: 1.0000
Epoch 119/200
3/3 [==============================] - 0s 12ms/step - loss: 0.0154 - accuracy: 1.0000 - val_loss: 0.0712 - val_accuracy: 0.9583
Epoch 120/200
3/3 [==============================] - 0s 12ms/step - loss: 0.1069 - accuracy: 0.9583 - val_loss: 0.1202 - val_accuracy: 0.9167
Epoch 121/200
3/3 [==============================] - 0s 13ms/step - loss: 0.0323 - accuracy: 0.9896 - val_loss: 0.0351 - val_accuracy: 1.0000
Epoch 122/200
3/3 [==============================] - 0s 12ms/step - loss: 0.0361 - accuracy: 0.9792 - val_loss: 0.0288 - val_accuracy: 1.0000
Epoch 123/200
3/3 [==============================] - 0s 14ms/step - loss: 0.0242 - accuracy: 0.9896 - val_loss: 0.0512 - val_accuracy: 1.0000
Epoch 124/200
3/3 [=

# 2. sklearn 이용
- 원핫인코딩을 하지 않고 라벨인코딩까지만 해야 작동

In [13]:
from sklearn.neural_network import MLPClassifier # 다층 퍼셉트론 분류기

In [14]:
# 1. 데이터
train_X.shape, train_y.shape, test_X.shape, test_y.shape

((120, 4), (120,), (30, 4), (30,))

In [15]:
model = MLPClassifier(
            hidden_layer_sizes=(68, 128, 50), # 은닉층의 units 개수
            activation='relu',
            solver='adam', # SGD등의 optimizer
            batch_size=40,
            max_iter=1000, # 학습 최대 횟수
            early_stopping=True, # 조기 종료 활성화 여부
            n_iter_no_change=10, # EarlyStopping의 patience와 유사
            validation_fraction=0.1, # = validation_split() : 검증셋 비율
            warm_start=False # True일 경우 이전 학습에 이어서 학습(W와 b값 이전 학습 결과치부터
)

In [16]:
# 학습
model.fit(train_X, train_y)

MLPClassifier(batch_size=40, early_stopping=True,
              hidden_layer_sizes=(68, 128, 50), max_iter=1000)

In [17]:
model.score(test_X, test_y)

0.8666666666666667

In [18]:
iris_X[0]

array([5.1, 3.5, 1.4, 0.2])

In [19]:
# 모델 사용
input_data = [[5.1, 3.5, 1.4, 0.2]]
model.predict(input_data) # argmax까지 해서 뿌려짐

array([0])

In [20]:
# 교차표
test_y

array([1, 0, 1, 1, 0, 2, 0, 2, 0, 0, 2, 2, 2, 1, 1, 1, 0, 2, 1, 0, 2, 2,
       0, 0, 2, 1, 2, 1, 1, 0])

In [22]:
y_hat = model.predict(test_X)
y_hat

array([1, 0, 1, 1, 0, 2, 0, 2, 0, 0, 2, 2, 2, 2, 1, 2, 0, 2, 2, 0, 2, 2,
       0, 0, 2, 1, 2, 2, 1, 0])

In [23]:
confusion_matrix(test_y, y_hat)

array([[10,  0,  0],
       [ 0,  6,  4],
       [ 0,  0, 10]], dtype=int64)

In [24]:
pd.crosstab(test_y, y_hat, rownames=['실제'], colnames=['예측'])

예측,0,1,2
실제,,,
0,10,0,0
1,0,6,4
2,0,0,10


# 3. 클래스를 생성하여 모델 생성함수 사용 

In [26]:
class DNNClassifier:
    @staticmethod
    # 모델 설정
    def build(input_dim=4, activation='relu', output_dim=3):
        model = Sequential([
            Input(input_dim),
            Dense(units=50, activation=activation),
            Dense(30, activation=activation),
            Dense(units=output_dim, activation='softmax')
        ])
    # 학습 설정
        model.compile(loss='sparse_categorical_crossentropy',
                      optimizer='adam',
                      metrics=['accuracy'])
        return model

In [29]:
# 1. 데이터
print(train_X.shape, train_y.shape, test_X.shape, test_y.shape)
# 2. 모델
model = DNNClassifier.build(input_dim=4, activation='elu')
# 3. 모델 학습
hist = model.fit(train_X, train_y, epochs=50, validation_split=0.1, verbose=0)

(120, 4) (120,) (30, 4) (30,)


In [30]:
# 모델 평가
loss, accuracy = model.evaluate(test_X, test_y)
print(f'정확도 : {accuracy * 100} %')

1/1 [==============================] - 0s 18ms/step - loss: 0.2373 - accuracy: 0.9000
정확도 : 89.99999761581421 %


# 4. 함수형 API
- 병렬처리, Residual block

In [35]:
# 기존 model1
model = Sequential()
model.add(Input(shape=(4,)))
model.add(Dense(units=50, activation='relu'))
model.add(Dense(units=30, activation='relu'))
model.add(Dense(units=3, activation='softmax'))
# 기존 model2
model = Sequential([
            Input(shape=(4,)),
            Dense(units=50, activation='relu'),
            Dense(30, activation='relu'),
            Dense(units=3, activation='softmax')
        ])
model.summary()

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_24 (Dense)            (None, 50)                250       
                                                                 
 dense_25 (Dense)            (None, 30)                1530      
                                                                 
 dense_26 (Dense)            (None, 3)                 93        
                                                                 
Total params: 1,873
Trainable params: 1,873
Non-trainable params: 0
_________________________________________________________________


In [31]:
# 기존 model3 (함수형 API 스타일)
from tensorflow.keras import Model
input_ = Input(shape=(4,))
layer1 = Dense(units=50, activation='relu')(input_)
layer2 = Dense(30, activation='relu')(layer1)
output = Dense(units=3, activation='softmax')(layer2)
model = Model(inputs=input_, outputs=output)
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_5 (InputLayer)        [(None, 4)]               0         
                                                                 
 dense_14 (Dense)            (None, 50)                250       
                                                                 
 dense_15 (Dense)            (None, 30)                1530      
                                                                 
 dense_16 (Dense)            (None, 3)                 93        
                                                                 
Total params: 1,873
Trainable params: 1,873
Non-trainable params: 0
_________________________________________________________________


In [39]:
# 병렬처리 방식
from tensorflow.keras.layers import concatenate
input_ = Input(shape=(4))
dense1 = Dense(units=50, activation='relu')(input_)
dense2 = Dense(80, activation='relu')(input_)
dense3 = Dense(30, activation='relu')(input_)
x = concatenate((dense1, dense2, dense3))
output = Dense(3, activation='softmax')(x)
model = Model(inputs=input_, outputs=output)
model.summary() # 4*50+50 + 4*80+80 + 4*30+30 + 160*3+3 = 1283

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_11 (InputLayer)          [(None, 4)]          0           []                               
                                                                                                  
 dense_30 (Dense)               (None, 50)           250         ['input_11[0][0]']               
                                                                                                  
 dense_31 (Dense)               (None, 80)           400         ['input_11[0][0]']               
                                                                                                  
 dense_32 (Dense)               (None, 30)           150         ['input_11[0][0]']               
                                                                                            

In [44]:
# Residual Block : 딥러닝에서 기울기 소실 문제로 학습이 잘 되지 않는 부분을 해결하기 위한 제안
from tensorflow.keras.layers import add
input_ = Input(shape=(4,))
dense1 = Dense(units=50, activation='relu')(input_)
dense2 = Dense(50, 'relu')(dense1)
dense3 = add((dense1, dense2))
output = Dense(3, 'softmax')(dense3)
model = Model(inputs=input_, outputs=output)
model.summary()

Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_14 (InputLayer)          [(None, 4)]          0           []                               
                                                                                                  
 dense_39 (Dense)               (None, 50)           250         ['input_14[0][0]']               
                                                                                                  
 dense_40 (Dense)               (None, 50)           2550        ['dense_39[0][0]']               
                                                                                                  
 add_1 (Add)                    (None, 50)           0           ['dense_39[0][0]',               
                                                                  'dense_40[0][0]']         

In [46]:
# 학습 과정 설정 & 학습
model.compile(loss='sparse_categorical_crossentropy',
             optimizer='adam', metrics=['accuracy'])
model.fit(train_X, train_y, epochs=200, verbose=1)

Epoch 1/200
4/4 [==============================] - 0s 2ms/step - loss: 0.0620 - accuracy: 0.9750
Epoch 2/200
4/4 [==============================] - 0s 2ms/step - loss: 0.0714 - accuracy: 0.9667
Epoch 3/200
4/4 [==============================] - 0s 1ms/step - loss: 0.0657 - accuracy: 0.9750
Epoch 4/200
4/4 [==============================] - 0s 2ms/step - loss: 0.0629 - accuracy: 0.9833
Epoch 5/200
4/4 [==============================] - 0s 2ms/step - loss: 0.0591 - accuracy: 0.9917
Epoch 6/200
4/4 [==============================] - 0s 1ms/step - loss: 0.0583 - accuracy: 0.9917
Epoch 7/200
4/4 [==============================] - 0s 1ms/step - loss: 0.0586 - accuracy: 0.9917
Epoch 8/200
4/4 [==============================] - 0s 2ms/step - loss: 0.0575 - accuracy: 0.9917
Epoch 9/200
4/4 [==============================] - 0s 2ms/step - loss: 0.0546 - accuracy: 0.9917
Epoch 10/200
4/4 [==============================] - 0s 2ms/step - loss: 0.0569 - accuracy: 0.9833
Epoch 11/200
4/4 [===========

4/4 [==============================] - 0s 2ms/step - loss: 0.0338 - accuracy: 0.9917
Epoch 85/200
4/4 [==============================] - 0s 1ms/step - loss: 0.0334 - accuracy: 0.9917
Epoch 86/200
4/4 [==============================] - 0s 1ms/step - loss: 0.0331 - accuracy: 0.9917
Epoch 87/200
4/4 [==============================] - 0s 1000us/step - loss: 0.0319 - accuracy: 0.9917
Epoch 88/200
4/4 [==============================] - 0s 1ms/step - loss: 0.0325 - accuracy: 0.9917
Epoch 89/200
4/4 [==============================] - 0s 1ms/step - loss: 0.0319 - accuracy: 0.9917
Epoch 90/200
4/4 [==============================] - 0s 2ms/step - loss: 0.0316 - accuracy: 0.9917
Epoch 91/200
4/4 [==============================] - 0s 1ms/step - loss: 0.0321 - accuracy: 0.9917
Epoch 92/200
4/4 [==============================] - 0s 2ms/step - loss: 0.0313 - accuracy: 0.9917
Epoch 93/200
4/4 [==============================] - 0s 1ms/step - loss: 0.0311 - accuracy: 0.9917
Epoch 94/200
4/4 [============

Epoch 167/200
4/4 [==============================] - 0s 1ms/step - loss: 0.0263 - accuracy: 0.9917
Epoch 168/200
4/4 [==============================] - 0s 2ms/step - loss: 0.0227 - accuracy: 0.9917
Epoch 169/200
4/4 [==============================] - 0s 1ms/step - loss: 0.0262 - accuracy: 0.9917
Epoch 170/200
4/4 [==============================] - 0s 1ms/step - loss: 0.0297 - accuracy: 0.9833
Epoch 171/200
4/4 [==============================] - 0s 1ms/step - loss: 0.0268 - accuracy: 0.9917
Epoch 172/200
4/4 [==============================] - 0s 1ms/step - loss: 0.0252 - accuracy: 0.9917
Epoch 173/200
4/4 [==============================] - 0s 2ms/step - loss: 0.0224 - accuracy: 1.0000
Epoch 174/200
4/4 [==============================] - 0s 2ms/step - loss: 0.0299 - accuracy: 0.9833
Epoch 175/200
4/4 [==============================] - 0s 2ms/step - loss: 0.0249 - accuracy: 0.9917
Epoch 176/200
4/4 [==============================] - 0s 2ms/step - loss: 0.0240 - accuracy: 0.9917
Epoch 177/

In [47]:
# 학습 평가
loss, accuracy = model.evaluate(test_X, test_y, verbose=0)
print(f'accuracy : {accuracy}')

accuracy : 0.9333333373069763
